# GIFT v2.2 - Variational G2 Metric Extraction

## Overview

This notebook implements a Physics-Informed Neural Network (PINN) to solve a **constrained variational problem** on G2 geometry.

**Key insight**: This is NOT a simulation of a pre-existing manifold. It is the numerical resolution of a minimization problem whose solution, if it exists, defines the geometry.

### Mathematical Formulation

Find $\phi \in \Lambda^3_+(\mathbb{R}^7)$ minimizing:
$$F[\phi] = ||d\phi||^2_{L^2} + ||d^*\phi||^2_{L^2}$$

Subject to GIFT v2.2 constraints:

| Constraint | Value | Origin |
|------------|-------|--------|
| $b_2$ | 21 | E8 decomposition |
| $b_3$ | 77 = 35 + 42 | Cohomology split |
| $\det(g)$ | 65/32 | Derived from $h^* = 99$ |
| $\kappa_T$ | 1/61 | Global torsion magnitude |
| $\phi$ positive | $\phi \in G_2$ cone | Valid G2 structure |

### Difference from Previous Approach

| Aspect | Old (TCS-based) | New (Variational) |
|--------|-----------------|-------------------|
| Starting point | "Construct TCS manifold" | "Define constraint system" |
| Role of (21,77) | "Verify these emerge" | "Impose as constraints" |
| Role of PINN | "Approximate known geometry" | "Solve optimization problem" |
| Success criterion | "Match TCS structure" | "Satisfy constraints + minimize F" |

## 1. Setup

In [ ]:
import sys
import os
from pathlib import Path

# Robust path setup for portability
# Works whether notebook is run from:
# - G2_ML/2_1/ directory
# - GIFT root directory
# - Google Colab / Binder

notebook_dir = Path('.').resolve()

# Find GIFT root by looking for characteristic files
def find_gift_root(start_path):
    """Find GIFT root directory by looking for README.md and G2_ML/"""
    current = Path(start_path).resolve()
    for _ in range(5):  # Max 5 levels up
        if (current / 'README.md').exists() and (current / 'G2_ML').exists():
            return current
        if current.parent == current:
            break
        current = current.parent
    return None

gift_root = find_gift_root(notebook_dir)

if gift_root is None:
    # Fallback: assume we're in G2_ML/2_1
    gift_root = notebook_dir.parent.parent
    print(f"Warning: GIFT root not found, assuming: {gift_root}")

# Add paths for imports
module_dir = gift_root / 'G2_ML' / '2_1'
if str(module_dir) not in sys.path:
    sys.path.insert(0, str(module_dir))
if str(gift_root) not in sys.path:
    sys.path.insert(0, str(gift_root))

print(f"GIFT root: {gift_root}")
print(f"Module dir: {module_dir}")

import torch
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Import v2.1 modules (direct import from local files)
from config import GIFTConfig, default_config
from g2_geometry import MetricFromPhi, G2Positivity, standard_phi_coefficients
from model import G2VariationalNet, HarmonicFormsNet
from training import Trainer, train_gift_g2, sample_coordinates
from validation import Validator, generate_validation_report

print("Modules loaded successfully.")
print(f"GIFT v2.2 targets: det(g)={default_config.det_g_target}, kappa_T={default_config.kappa_T:.6f}")

## 2. GIFT v2.2 Configuration

All values are **topologically determined** from:
- E8 x E8 gauge group (dimension 496)
- K7 manifold with G2 holonomy

No continuous adjustable parameters.

In [ ]:
config = GIFTConfig()

print("GIFT v2.2 Structural Parameters")
print("="*50)
print(f"")
print(f"Cohomology:")
print(f"  b2(K7) = {config.b2_K7}")
print(f"  b3(K7) = {config.b3_K7} = {config.b3_local} + {config.b3_global}")
print(f"  h* = {config.h_star}")
print(f"")
print(f"Derived Constants:")
print(f"  det(g) = {config.det_g_target} = 65/32")
print(f"  kappa_T = {config.kappa_T:.6f} = 1/61")
print(f"  sin^2(theta_W) = {config.sin2_theta_W:.6f} = 3/13")
print(f"  tau = {config.tau:.6f} = 3472/891")
print(f"")
print(f"Training:")
print(f"  Total epochs: {config.total_epochs}")
print(f"  Phases: {len(config.phases)}")
for i, phase in enumerate(config.phases):
    print(f"    Phase {i+1}: {phase['name']} ({phase['epochs']} epochs)")

## 3. Standard G2 Structure

Before training, let's examine the standard G2 3-form $\phi_0$.

In [ ]:
# Standard G2 3-form
phi_0 = standard_phi_coefficients()

print("Standard G2 3-form phi_0:")
print(f"Components: {phi_0.shape[0]}")
print(f"Non-zero entries: {(phi_0.abs() > 0.5).sum().item()}")
print(f"")
print("Non-zero components:")
for i, val in enumerate(phi_0):
    if abs(val) > 0.5:
        print(f"  phi[{i}] = {val:+.0f}")

In [ ]:
# Metric from standard phi
metric_extractor = MetricFromPhi()
g_standard = metric_extractor(phi_0)

print("Metric from standard G2:")
print(g_standard.numpy().round(4))
print(f"")
print(f"det(g) = {torch.det(g_standard).item():.6f}")
print(f"Eigenvalues: {torch.linalg.eigvalsh(g_standard).numpy().round(4)}")

## 4. Model Architecture

The `G2VariationalNet` uses:
- Fourier features for smooth positional encoding
- Deep MLP with LayerNorm and SiLU activation
- Output: 35 independent 3-form components
- Optional projection to G2 positive cone

In [ ]:
# Create model
model = G2VariationalNet(config).to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Model Architecture")
print("="*50)
print(f"Total parameters: {n_params:,}")
print(f"Trainable parameters: {n_trainable:,}")
print(f"")
print(f"Fourier features: {config.fourier_features}")
print(f"Hidden dimension: {config.hidden_dim}")
print(f"Layers: {config.n_layers}")
print(f"Output dimension: {config.n_phi_components}")

In [ ]:
# Test forward pass
x_test = sample_coordinates(100, device)
phi_test = model(x_test)

print(f"Input shape: {x_test.shape}")
print(f"Output shape: {phi_test.shape}")
print(f"")
print(f"Output stats:")
print(f"  Mean: {phi_test.mean().item():.4f}")
print(f"  Std: {phi_test.std().item():.4f}")
print(f"  Min: {phi_test.min().item():.4f}")
print(f"  Max: {phi_test.max().item():.4f}")

## 5. Training

The training proceeds in 4 phases:
1. **Initialization**: Establish valid G2 structure
2. **Constraint satisfaction**: Achieve det(g) = 65/32
3. **Torsion targeting**: Achieve kappa_T = 1/61
4. **Cohomology refinement**: Refine (b2, b3) = (21, 77)

In [ ]:
# Create output directory
output_dir = Path('outputs_v2_1')
output_dir.mkdir(exist_ok=True)

print(f"Output directory: {output_dir.resolve()}")

In [ ]:
# Quick training (reduced epochs for demonstration)
# For full training, use config with total_epochs=10000

demo_config = GIFTConfig(
    total_epochs=2000,
    phases=[
        {"name": "initialization", "epochs": 500, "focus": "establish G2",
         "weights": {"torsion": 1.0, "det": 0.5, "positivity": 2.0, "cohomology": 0.1}},
        {"name": "constraint", "epochs": 700, "focus": "det(g)",
         "weights": {"torsion": 1.0, "det": 2.0, "positivity": 1.0, "cohomology": 0.5}},
        {"name": "torsion", "epochs": 500, "focus": "kappa_T",
         "weights": {"torsion": 3.0, "det": 1.0, "positivity": 1.0, "cohomology": 1.0}},
        {"name": "refinement", "epochs": 300, "focus": "cohomology",
         "weights": {"torsion": 2.0, "det": 1.0, "positivity": 0.5, "cohomology": 2.0}},
    ]
)

print("Training with reduced epochs for demonstration.")
print(f"Total: {demo_config.total_epochs} epochs")

In [ ]:
# Train model
model, history = train_gift_g2(
    config=demo_config,
    device=device,
    save_path=output_dir,
    verbose=True
)

## 6. Training Analysis

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Loss
ax = axes[0, 0]
ax.semilogy(history.loss_history)
for boundary in history.phase_boundaries:
    ax.axvline(boundary, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.3)

# Determinant
ax = axes[0, 1]
ax.plot(history.det_history)
ax.axhline(config.det_g_target, color='g', linestyle='--', label=f'Target: {config.det_g_target}')
for boundary in history.phase_boundaries:
    ax.axvline(boundary, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('det(g)')
ax.set_title('Metric Determinant')
ax.legend()
ax.grid(True, alpha=0.3)

# Torsion
ax = axes[1, 0]
ax.plot(history.torsion_history)
ax.axhline(config.kappa_T, color='g', linestyle='--', label=f'Target: 1/61')
for boundary in history.phase_boundaries:
    ax.axvline(boundary, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('kappa_T')
ax.set_title('Torsion Magnitude')
ax.legend()
ax.grid(True, alpha=0.3)

# Positivity
ax = axes[1, 1]
ax.semilogy([p + 1e-10 for p in history.positivity_history])
for boundary in history.phase_boundaries:
    ax.axvline(boundary, color='r', linestyle='--', alpha=0.5)
ax.set_xlabel('Epoch')
ax.set_ylabel('Positivity Violation')
ax.set_title('Positivity Constraint')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'training_curves.png', dpi=150)
plt.show()

## 7. Validation

In [ ]:
# Run validation
report = generate_validation_report(
    model,
    config=config,
    device=device,
    save_path=output_dir / 'validation_report.json'
)

## 8. Geometric Analysis

In [ ]:
# Sample points and compute metrics
model.eval()
n_samples = 5000

with torch.no_grad():
    x = sample_coordinates(n_samples, device)
    phi, g = model.get_phi_and_metric(x)
    det_g = torch.det(g)
    eigenvalues = torch.linalg.eigvalsh(g)

In [ ]:
# Determinant distribution
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# det(g) histogram
ax = axes[0]
det_np = det_g.cpu().numpy()
ax.hist(det_np, bins=50, density=True, alpha=0.7)
ax.axvline(config.det_g_target, color='r', linestyle='--', 
           label=f'Target: {config.det_g_target:.4f}')
ax.axvline(det_np.mean(), color='g', linestyle='--', 
           label=f'Mean: {det_np.mean():.4f}')
ax.set_xlabel('det(g)')
ax.set_ylabel('Density')
ax.set_title('Metric Determinant Distribution')
ax.legend()

# Minimum eigenvalue
ax = axes[1]
min_eig = eigenvalues[:, 0].cpu().numpy()
ax.hist(min_eig, bins=50, density=True, alpha=0.7)
ax.axvline(0, color='r', linestyle='--', label='Zero')
ax.set_xlabel('Minimum eigenvalue of g')
ax.set_ylabel('Density')
ax.set_title('Positivity Check')
ax.legend()

# Eigenvalue spectrum
ax = axes[2]
for i in range(7):
    eig_i = eigenvalues[:, i].cpu().numpy()
    ax.violinplot([eig_i], positions=[i], showmeans=True, showmedians=True)
ax.set_xlabel('Eigenvalue index')
ax.set_ylabel('Value')
ax.set_title('Metric Eigenvalue Spectrum')

plt.tight_layout()
plt.savefig(output_dir / 'metric_analysis.png', dpi=150)
plt.show()

In [ ]:
# Summary statistics
print("Metric Statistics")
print("="*50)
print(f"det(g):")
print(f"  Mean: {det_np.mean():.6f}")
print(f"  Std:  {det_np.std():.6f}")
print(f"  Target: {config.det_g_target:.6f}")
print(f"  Error: {100*abs(det_np.mean() - config.det_g_target)/config.det_g_target:.2f}%")
print(f"")
print(f"Eigenvalues:")
for i in range(7):
    eig_i = eigenvalues[:, i].cpu().numpy()
    print(f"  lambda_{i}: {eig_i.mean():.4f} +/- {eig_i.std():.4f}")
print(f"")
print(f"Positivity: {(min_eig > 0).mean()*100:.1f}% of samples have g > 0")

## 9. Mathematical Framing

The output of this computation should be understood as:

> **Theorem (Conditional Existence)**
> 
> Let P be the variational problem: minimize $||d\phi||^2 + ||d^*\phi||^2$ over G2 3-forms
> subject to $\det(g(\phi)) = 65/32$ and $\kappa_T = 1/61$.
> 
> The PINN produces $\phi_{\text{num}}$ satisfying:
> - $F[\phi_{\text{num}}] < \epsilon$ (torsion bound)
> - $|\det(g) - 65/32| < \delta_1$
> - $|\kappa_T - 1/61| < \delta_2$
> 
> If $\epsilon$ is sufficiently small, by G2 deformation theory (Joyce), there exists
> an exact torsion-free G2 structure $\phi_{\text{exact}}$ with $||\phi_{\text{exact}} - \phi_{\text{num}}|| = O(\epsilon)$.

This positions the work as:
- **Numerical evidence** for existence of a specific G2 geometry
- **Not** a claim to have constructed the manifold rigorously
- **Invitation** to mathematicians: prove this geometry exists

## 10. Export Artifacts

In [ ]:
# Save trained model and artifacts
torch.save({
    'model_state': model.state_dict(),
    'config': {
        'det_g_target': config.det_g_target,
        'kappa_T': config.kappa_T,
        'b2_K7': config.b2_K7,
        'b3_K7': config.b3_K7,
        'tau': config.tau,
    },
    'validation': report,
}, output_dir / 'gift_g2_v2_1.pt')

print(f"Model saved to {output_dir / 'gift_g2_v2_1.pt'}")

In [ ]:
# Export sample points and metrics for further analysis
np.savez(
    output_dir / 'samples.npz',
    coords=x.cpu().numpy(),
    phi=phi.cpu().numpy(),
    metric=g.cpu().numpy(),
    det_g=det_g.cpu().numpy(),
    eigenvalues=eigenvalues.cpu().numpy(),
)

print(f"Samples saved to {output_dir / 'samples.npz'}")

## Summary

This notebook demonstrates the variational approach to G2 metric extraction for GIFT v2.2:

1. **Constraints are primary**: We impose det(g) = 65/32, kappa_T = 1/61 from GIFT theory
2. **Geometry is emergent**: The metric and 3-form emerge from solving the optimization
3. **No TCS assumption**: We don't assume a specific construction method
4. **Numerical evidence**: The solution provides evidence for existence of this geometry

### Next Steps

1. Train with full 10000 epochs for better convergence
2. Compute harmonic forms to verify (b2, b3) = (21, 77)
3. Analyze stability under deformations
4. Compute Yukawa tensor and verify tau = 3472/891